In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
from sklearn.cluster import KMeans


import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv(r'C:\Users\martin.olivares\Desktop\projects\best-route\test_2.csv')
data.fillna(method='ffill', inplace=True)

df=pd.DataFrame()
df['address']=data['Direccion de inicio']
df['recogida']=data['Hora de recogida']

df.fillna(method='ffill', inplace=True)



In [3]:

#Función para verificar que las direcciones estén correctas. Posteriormente se dejan fuera las incorrectas (momentaneo)

def correct_address(address):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(address)
    if location:
        return location.address
    else:
        return 'no_existe_la_direccion'


df['corrected_address'] = df['address'].apply(correct_address)

filter=df['corrected_address']!='no_existe_la_direccion'
filter2= df['address']!='PASAJE CERO 14 CHORRILLOS, VIÑA DEL MAR'
df.where(filter & filter2,inplace=True)
df.dropna(inplace=True)

df


,address,recogida,corrected_address
0,"DOCTOR ISRAEL ROIZBLATT 130, VALPARAISO",6:20,"Doctor Israel Roizblatt, Población Dr. Israel ..."
1,"SEPTIMO DE LINEA 17, VALPARAÍSO",6:20,"Séptimo de Línea, Villa El Totoral, San Felipe..."
2,"Dionisio Hernández 450,Viña del Mar",6:30,"Dionisio Hernández, Población Nuevo Horizonte,..."
3,"Camila 109, Valparaíso",6:50,"109, Camila, Cerro San Juan de Dios, Almendral..."
4,"Ramaditas 1626, Valparaíso",20:40,"Ramaditas, Cerro Santa Elena, San Roque, Valpa..."
5,"Navío San Martín 70, Valparaíso",20:40,"Navío San Martín, Bosque Inglés, San Roque, Va..."
6,"Viana 1155, Valparaíso, Viña del Mar, Valparaíso",20:40,"Viana, Población Saenz, Forestal, Viña del Mar..."
8,"Irma Johnson Molina 2579, Quilpué, Valparaíso",20:30,"Altos de Marga Marga I, 2579, Irma Johnson Mol..."
9,"3 ORIENTE 1270, VIÑA DEL MAR",20:30,"3 Oriente, Población Británica, Viña del Mar, ..."


In [4]:
#crea coordenadas geográficas
geolocator = Nominatim(user_agent="geoapiExercises")

# Calcula la distancia entre cada par de direcciones
distances = []
for i in range(len(df)):
    address1 = df.iloc[i]['address']
    loc1 = geolocator.geocode(address1)
    lat1, lon1 = loc1.latitude, loc1.longitude
    point1 = (lat1, lon1)
    for j in range(i+1, len(df)):
        address2 = df.iloc[j]['address']
        loc2 = geolocator.geocode(address2)
        lat2, lon2 = loc2.latitude, loc2.longitude
        point2 = (lat2, lon2)
        distances.append((i, j, great_circle(point1, point2).m))

# Crea una matriz con las distancias
X = np.zeros((len(df), len(df)))
for i, j, d in distances:
    X[i, j] = d
    X[j, i] = d

In [5]:
# Inicializa el modelo KMeans con n_clusters=1
kmeans = KMeans(n_clusters=1)

# Ajusta el modelo a la matriz
kmeans.fit(X)

# Asigna etiquetas a cada punto de datos
df['label'] = kmeans.labels_

# Continúa agrupando hasta que cada clúster tenga 4 o menos direcciones
while df.groupby('label').agg({'address':'count'}).max().values[0] > 4:
    kmeans.n_clusters += 1
    kmeans.fit(X)
    df['label'] = kmeans.labels_




# Verifica los resultados
print(df.groupby('label').agg({'address':'count'}))

df

       address
label         
0            4
1            1
2            3
3            1


,address,recogida,corrected_address,label
0,"DOCTOR ISRAEL ROIZBLATT 130, VALPARAISO",6:20,"Doctor Israel Roizblatt, Población Dr. Israel ...",0
1,"SEPTIMO DE LINEA 17, VALPARAÍSO",6:20,"Séptimo de Línea, Villa El Totoral, San Felipe...",1
2,"Dionisio Hernández 450,Viña del Mar",6:30,"Dionisio Hernández, Población Nuevo Horizonte,...",2
3,"Camila 109, Valparaíso",6:50,"109, Camila, Cerro San Juan de Dios, Almendral...",0
4,"Ramaditas 1626, Valparaíso",20:40,"Ramaditas, Cerro Santa Elena, San Roque, Valpa...",0
5,"Navío San Martín 70, Valparaíso",20:40,"Navío San Martín, Bosque Inglés, San Roque, Va...",0
6,"Viana 1155, Valparaíso, Viña del Mar, Valparaíso",20:40,"Viana, Población Saenz, Forestal, Viña del Mar...",2
8,"Irma Johnson Molina 2579, Quilpué, Valparaíso",20:30,"Altos de Marga Marga I, 2579, Irma Johnson Mol...",3
9,"3 ORIENTE 1270, VIÑA DEL MAR",20:30,"3 Oriente, Población Británica, Viña del Mar, ...",2
